In [ ]:
#Thickness Perturbation
import numpy as np
import os
import json
def ThickP(inp_file,output_file):
    # Define input and output file paths
    #print(inp_file)
    #print(output_file)
    # Define radius parameters
    mean_radius = 1.0  # Mean beam radius
    std_dev = 0.3       # Standard deviation for variation
    min_radius = 0.75    # Minimum allowed radius
    max_radius = 1.25    # Maximum allowed radius

    # Read input file
    with open(inp_file, "r") as file:
        lines = file.readlines()

    # Extract element IDs and their node connections
    updated_lines = []
    elset_definitions = []
    nset_definitions = []
    beam_section_definitions = []
    reading_elements = False
    section_found = False
    reading_beam_section = False
    section_index = None  # To track where *Section is found

    element_nodes = {}  # Dictionary to store element-to-node mapping
    element_data=[]
    for i, line in enumerate(lines):
        # Detect start of elements section
        if "*Element" in line:
            reading_elements = True
            updated_lines.append(line.strip())
            continue

        # Stop reading elements when encountering another keyword
        if reading_elements and "*" in line and "Element" not in line:
            reading_elements = False

        if reading_beam_section:
            if "*" in line and "Beam Section" not in line:  # End of section
                reading_beam_section = False
            continue  # Skip old section lines

        # Read elements only within the *Element section
        if reading_elements:
            parts = [p.strip() for p in line.split(",") if p.strip()]

            if len(parts) < 3:  # Ensure at least an element ID and two nodes
                continue

            if not parts[0].isdigit():  # Skip non-numeric lines
                continue

            # Extract element ID and node IDs
            element_id = int(parts[0])
            node_1 = int(parts[1])
            node_2 = int(parts[2])
            element_nodes[element_id] = (node_1, node_2)  # Store node pair

            # Store element definition
            updated_lines.append(line.strip())
        else:
            updated_lines.append(line.strip())

        if "** Section" in line and not section_found:
            section_found = True
            section_index = i  # Store the line index of *Section

    # Generate `*Elset` and `*Nset` definitions 
    for element_id, (node_1, node_2) in element_nodes.items():
        nset_definitions.append(f"*Nset, nset=_PickedSet{element_id}, internal")
        if node_1<node_2:
            nset_definitions.append(f"{node_1}, {node_2}")
        if node_1>node_2:
            nset_definitions.append(f"{node_2}, {node_1}")
        elset_definitions.append(f"*Elset, elset=_PickedSet{element_id}, internal")
        elset_definitions.append(f"{element_id},")

    # Generate `*Beam Section` definitions 
    for element_id in element_nodes.keys():
        radius = np.clip(np.random.normal(mean_radius, std_dev), min_radius, max_radius)
        element_data.append(float(radius))
        beam_section_definitions.append(f"** Section: Section-{element_id} Profile: Profile-{element_id}")
        beam_section_definitions.append(f"*Beam Section, elset=_PickedSet{element_id}, material=Material-1, temperature=GRADIENTS, section=CIRC")
        beam_section_definitions.append(f"{radius:.6f}")
        beam_section_definitions.append("0., 0., -1.")

    # Insert `*Elset` and `*Nset` 
    if section_index is not None:
        updated_lines = updated_lines[:section_index] + nset_definitions+ elset_definitions + beam_section_definitions + updated_lines[section_index+4:]



    # Write modified .inp file
    with open(output_file, "w") as file:
        for line in updated_lines:
            file.write(line + "\n")

    print(f"Modified .inp file saved as {output_file}")

    if os.path.exists("thickness.txt"):
        with open("thickness.txt", "r") as txtfile:
            try:
                all_iterations = json.load(txtfile)  # Load existing data
            except json.JSONDecodeError:
                all_iterations = []  # If file is empty, start fresh
    else:
        all_iterations = []  # First iteration

    # Append new iteration
    all_iterations.append(element_data)
    with open("thickness.txt", "w") as txtfile:
        txtfile.write("[\n")  # Start the outer list

        for i, iteration_data in enumerate(all_iterations):
            formatted_nodes = ", ".join([f"{r:.6f}" for r in iteration_data])
            txtfile.write(f"  [{formatted_nodes}]")  # Write each iteration's nodes in one line

            # Add a comma after each iteration except the last one
            if i < len(all_iterations) - 1:
                txtfile.write(",\n")
            else:
                txtfile.write("\n")

        txtfile.write("]\n")  # End the outer list

    print(" Thickness saved to thickness.txt")